In [2]:
import pandas as pd 
import numpy as np
import requests

In [15]:
# read my 1+ StreamingHistory files (depending on how)
df_stream0 = pd.read_json('StreamingHistory0.json')
df_stream1 = pd.read_json('StreamingHistory1.json')

# merge streaming dataframes
df_stream = pd.concat([df_stream0,df_stream1])

# create a 'uniqueID' for each song by combining the fields 'artistName' and 'trackName'
df_stream['UniqueID'] = df_stream['artistName']+":"+df_stream['trackName']

df_stream.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2022-07-09 12:19,Kygo,Higher Love,11730,Kygo:Higher Love
1,2022-07-09 12:19,Drake,Falling Back,32810,Drake:Falling Back
2,2022-07-10 09:00,Dirtyphonics,Stage Divers - Original Mix,22506,Dirtyphonics:Stage Divers - Original Mix
3,2022-07-10 09:04,Lorde,Supercut,277506,Lorde:Supercut
4,2022-07-10 09:29,Kygo,Stole the Show,143497,Kygo:Stole the Show


In [4]:
# # read my edited Library json file into a pandas dataframe
# PATH_LIB = '/home/mijin/code/data-pipeline/dags/hive/mijin/spotify/YourLibrary.json'
# df_library = pd.read_json(PATH_LIB)

# # add UniqueID column (same as above)
# df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']

# # add column with track URI stripped of 'spotify:track:'
# new = df_library["uri"].str.split(":", expand=True)
# df_library['track_uri'] = new[2]

# df_library.head()


In [5]:
import json
import os
PATH_LIB = '/home/mijin/code/data-pipeline/dags/hive/mijin/spotify/YourLibrary.json'

# read the json file
with open(PATH_LIB, 'r') as file:
    data = json.load(file)

# function to recursively print all keys
def extract_unique_keys(data):
    unique_keys = set()
    if isinstance(data, dict):
        for key, value in data.items():
             unique_keys.add(key)
             unique_keys.update(extract_unique_keys(value))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, dict):
                unique_keys.update(extract_unique_keys(item))
    return unique_keys

# extract unique keys within each first-level key
unique_keys_per_key = {}
for key, value in data.items():
    unique_keys = extract_unique_keys(value)
    unique_keys_per_key[key] = unique_keys

# print the unique keys within each first-level key
for key, unique_keys in unique_keys_per_key.items():
    print(f"Unique keys in '{key}' :")
    for unique_key in unique_keys:
        print(unique_key)
    print()

Unique keys in 'tracks' :
uri
artist
album
track

Unique keys in 'albums' :
uri
album
artist

Unique keys in 'shows' :
publisher
uri
name

Unique keys in 'episodes' :
show
uri
name

Unique keys in 'bannedTracks' :

Unique keys in 'artists' :
uri
name

Unique keys in 'bannedArtists' :

Unique keys in 'other' :



In [6]:
# create a directory to store the divided datasets
output_dir = './divided_datasets'
os.makedirs(output_dir, exist_ok=True)

# divide the json dataset along the first-level key
for key, value in data.items():
    output_path = os.path.join(output_dir, f'{key}.json')
    with open(output_path, 'w') as output_file:
        json.dump(value, output_file, indent=4)

    print(f"Dataset for key '{key}' is saved to '{output_path}'")


Dataset for key 'tracks' is saved to './divided_datasets/tracks.json'
Dataset for key 'albums' is saved to './divided_datasets/albums.json'
Dataset for key 'shows' is saved to './divided_datasets/shows.json'
Dataset for key 'episodes' is saved to './divided_datasets/episodes.json'
Dataset for key 'bannedTracks' is saved to './divided_datasets/bannedTracks.json'
Dataset for key 'artists' is saved to './divided_datasets/artists.json'
Dataset for key 'bannedArtists' is saved to './divided_datasets/bannedArtists.json'
Dataset for key 'other' is saved to './divided_datasets/other.json'


In [7]:
# read my edited track_library json file into a pandas dataframe
df_library = pd.read_json('./divided_datasets/tracks.json')

# add UniqueID column (same as above)
df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']

# add column with track URI stripped of 'spotify:track:'
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]

df_library.head()

,artist,album,track,uri,UniqueID,track_uri
0,John De Sohn,Looking Up,Looking Up,spotify:track:4GvHfRUyNfNz86FbnDThDl,John De Sohn:Looking Up,4GvHfRUyNfNz86FbnDThDl
1,Ouai Stéphane,Ouai ouai,Bonjour c'est Stéphane encore,spotify:track:6q79iFTPE7eLM0Z7EEBQEV,Ouai Stéphane:Bonjour c'est Stéphane encore,6q79iFTPE7eLM0Z7EEBQEV
2,Diplo,Diplo Presents Thomas Wesley: Chapter 1 - Snak...,Lonely (with Jonas Brothers),spotify:track:6eXFl418w8hsxKcLgv1jwJ,Diplo:Lonely (with Jonas Brothers),6eXFl418w8hsxKcLgv1jwJ
3,Boombox Cartel,All I Want,All I Want,spotify:track:1TFljRRw6jLg0qGP1Qa1Gc,Boombox Cartel:All I Want,1TFljRRw6jLg0qGP1Qa1Gc
4,Caroline Polachek,Pang,So Hot You're Hurting My Feelings,spotify:track:5B6Kjha6RRIMWGN7zGsAaT,Caroline Polachek:So Hot You're Hurting My Fee...,5B6Kjha6RRIMWGN7zGsAaT


In [8]:
# create final dict as a copy df_stream
df_tableau = df_stream.copy()

# add column checking if streamed song is in library
# not used in this project but could be helpful for cool visualizations
df_tableau['InLibrary'] = np.where(df_tableau['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)

# left join with df_library on UniqueID to bring in album and track_uri
df_tableau = pd.merge(df_tableau, df_library[['album', 'UniqueID', 'track_uri']], how='left', on=['UniqueID'])

df_tableau.head(50)

,endTime,artistName,trackName,msPlayed,UniqueID,InLibrary,album,track_uri
0,2022-07-09 12:19,Kygo,Higher Love,11730,Kygo:Higher Love,0,NaN,NaN
1,2022-07-09 12:19,Drake,Falling Back,32810,Drake:Falling Back,0,NaN,NaN
2,2022-07-10 09:00,Dirtyphonics,Stage Divers - Original Mix,22506,Dirtyphonics:Stage Divers - Original Mix,0,NaN,NaN
3,2022-07-10 09:04,Lorde,Supercut,277506,Lorde:Supercut,0,NaN,NaN
4,2022-07-10 09:29,Kygo,Stole the Show,143497,Kygo:Stole the Show,0,NaN,NaN
5,2022-07-10 09:32,Mr. Oizo,Positif,171120,Mr. Oizo:Positif,0,NaN,NaN
6,2022-07-10 09:36,Mr. Oizo,The Church,212681,Mr. Oizo:The Church,0,NaN,NaN
7,2022-07-10 09:40,Gater,Taboo,259000,Gater:Taboo,1,Taboo (feat. Chesty Morganstein),5CVOeNkatHhW7Uc7M7zffK
8,2022-07-10 09:45,Ouai Stéphane,Ouai ouai,250909,Ouai Stéphane:Ouai ouai,0,NaN,NaN
9,2022-07-10 09:48,Handbraekes,Callgurls,195037,Handbraekes:Callgurls,0,NaN,NaN


In [9]:
# save my IDs from new project in Spotify Developer Dashboard
CLIENT_ID = 'f014be6f3056430ba3a886bab188611a'
CLIENT_SECRET = 'e2b7838194cb4fb8b9b6f1f6da57ab2f'

# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret' : CLIENT_SECRET 
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'



In [10]:
# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()

# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:

    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

# convert dictionary into dataframe with track_uri as the first column
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre.head(10)


,track_uri,artist_uri,genres
0,4GvHfRUyNfNz86FbnDThDl,3TVjdmhfFoYJLgBEkkZTBm,"[deep pop edm, pop dance, swedish tropical house]"
1,6q79iFTPE7eLM0Z7EEBQEV,2PLiFvOw5jPlHoGoxjeeaf,[]
2,6eXFl418w8hsxKcLgv1jwJ,5fMUXHkw8R8eOP2RNVYEZX,"[edm, electro house, moombahton, pop dance]"
3,1TFljRRw6jLg0qGP1Qa1Gc,4m1yRHUMhvB8gKAJTjK4kO,"[brostep, electro house, electronic trap]"
4,5B6Kjha6RRIMWGN7zGsAaT,4Ge8xMJNwt6EEXOzVXju9a,"[art pop, escape room, indie pop, indietronica]"
5,324Jd5N9usVLnRluow8ojb,6vqMDhoigg5btfdPsPTVFt,"[australian indie, australian indie rock, bris..."
6,05IyD7K9Q4lgu8W4Xfi9Lx,1ikID9RZZMvkuBGDWrqajq,"[future bass, pop edm, swedish electropop, vap..."
7,3unpulZnIiKRZ4MCymHfXx,63XBtGSEZINSyXylZxEUbv,[alternative r&b]
8,5d8a5B6RlYnquvR4X4OIrX,69GGBxA162lTqCwzJG5jLp,"[electropop, pop]"
9,2CgOd0Lj5MuvOqzqdaAXtS,3dz0NnIZhtKKeXZxLOxCam,"[complextro, edm, electro house, pop dance, pr..."


In [11]:
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

,track_uri,artist_uri,genres
0,4GvHfRUyNfNz86FbnDThDl,3TVjdmhfFoYJLgBEkkZTBm,deep pop edm
0,4GvHfRUyNfNz86FbnDThDl,3TVjdmhfFoYJLgBEkkZTBm,pop dance
0,4GvHfRUyNfNz86FbnDThDl,3TVjdmhfFoYJLgBEkkZTBm,swedish tropical house
1,6q79iFTPE7eLM0Z7EEBQEV,2PLiFvOw5jPlHoGoxjeeaf,NaN
2,6eXFl418w8hsxKcLgv1jwJ,5fMUXHkw8R8eOP2RNVYEZX,edm


In [12]:
# save df_tableau and df_genre_expanded as csv files that we can visualization
df_tableau.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpandedTable.csv')

print('done')

done
